<h1>K-means クラスタリング モデルの作成</h1>

BigQuery ML は教師なし学習をサポートしています。データをクラスタにグループ化するために [K-means](https://en.wikipedia.org/wiki/K-means_clustering) アルゴリズムを適用することができます。教師あり機械学習が予測分析を目的としているのとは異なり、教師なし学習では記述分析が目的となります。データドリブンの意思決定を下せるように、データを理解することが重要になります。

このチュートリアルでは、BigQuery ML の K 平均法モデルを使用して、[ロンドンのレンタル自転車の一般公開データセット](https://console.cloud.google.com/marketplace/details/greater-london-authority/london-bicycles?filter=solution-type:dataset&id=95374cac-2834-4fa2-a71f-fc033ccb5ce4&_ga=2.161383807.336887729.1589073706-431044393.1585885350&_gac=1.226746991.1587955074.CjwKCAjw4pT1BRBUEiwAm5QuR4lYhZuwjdvkKXaQcmeIKwclqw8o3ZXPRBCThiC8R0r-EKgVvMDCJhoCbAwQAvD_BwE)からデータのクラスタを構築します。同データセットには、ロンドンの Santander Cycle Hire Scheme の 2011 年から現在までの利用者数が含まれています。また、開始時と終了時のタイムスタンプ、ステーション名、乗車時間も含まれます。

このチュートリアルのクエリでは、BigQuery GIS で使用できる[地理関数](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions)を使用します。BigQuery GIS の詳細については、[BigQuery GIS](https://cloud.google.com/bigquery/docs/gis-intro) の概要をご覧ください。

<h2>目標</h2>

このチュートリアルの内容は次のとおりです。

- K 平均法クラスタリング モデルの作成
- BigQuery ML のクラスタの可視化に基づいたデータドリブンの意思決定

<h2>はじめに</h2>

データには、自然なグループやデータのクラスタが含まれている場合があります。データドリブンの意思決定が行えるように、これらのグループを記述的に識別することをおすすめします。たとえば小売業者であれば、似たような購買習慣を持つ顧客や同一地域の顧客を自然なグループとして識別したい場合があります。このようなプロセスを「顧客セグメンテーション」と呼びます。

顧客セグメンテーションを行う際に使用するデータには、訪問した店舗、購入した商品、支払った金額などがあります。モデルを作成して、このような顧客特性を反映したグループについて把握することで、そのグループを構成するユーザーにアピールする商品を考案できるようになります。

購入した商品の中から商品グループを見つけることもできます。この場合、商品の購入者、購入時期、購入元などに基づいて商品をクラスタ化します。モデルを作成して商品グループの特性を判別することで、クロスセルの改善方法について情報に基づいた決定を下せるようになります。

このチュートリアルでは、ロンドンのレンタル自転車のデータを自転車ステーションの属性に基づいてクラスタ化します。その際、BigQuery ML を使用して K-meansモデルを作成します。

<h2>ステップ 1: データセットを作成する</h2>

最初のステップとして、モデルを格納する BigQuery データセットを作成します。

In [ ]:
!bq mk --location eu bqml_tutorial_eu

<h2>ステップ 2: トレーニング データを確認する</h2>

次に、K-means モデルのトレーニングに使用するデータの確認を行います。このチュートリアルでは、以下の属性に基づいて自転車ステーションのクラスタリングを行います。

- レンタル期間
- 1 日あたりの利用数
- 市中心部からの距離

次のクエリで、トレーニング データをまとめます。このクエリは、チュートリアルの後半で `CREATE MODEL` ステートメントで使われます。

In [ ]:
%%bigquery df
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  *
FROM
  stationstats
ORDER BY
  distance_from_city_center ASC

In [ ]:
df.head(10)

<h3>クエリの詳細</h3>

このクエリは、`start_station_name` と `duration` を含む自転車レンタルのデータを抽出し、`distance-from-city-center` を含むステーション情報に結合します。そして、平均乗車時間と利用数を含んだ `stationstats` でステーションの属性を計算します。この計算には、ステーション属性 `distance_from_city_center` も含まれます。

このクエリでは、WITH 句を使用してサブクエリが定義されています。また、BigQuery GIS 関数 `ST_DISTANCE` と `ST_GEOGPOINT` も使用しています。

<h3>ステップ 3: K 平均法モデルを作成する</h3>

トレーニング データを確認したら、次のステップでは、そのデータを使用して K-means モデルを作成します。

`CREATE MODEL` ステートメントを `model_type=kmeans` オプション付きで使用することで、K-means モデルの作成とトレーニングが行えます。次のクエリでは、上記のクエリに `CREATE MODEL` ステートメントを追加し、データ内の `id` フィールドを削除します。

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
  bqml_tutorial_eu.london_station_clusters OPTIONS(model_type='kmeans',
    num_clusters=4) AS
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
  IF
    (EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 1
      OR EXTRACT(DAYOFWEEK
      FROM
        h.start_date) = 7,
      "weekend",
      "weekday") AS isweekday,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    isweekday,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name, isweekday)
SELECT
  * EXCEPT(station_name, isweekday)
FROM
  stationstats

<h3>クエリの詳細</h3>

`CREATE MODEL` ステートメントでは、必要なクラスタ数（4）を指定します。`station_name` は特徴量ではないため、`SELECT` ステートメントで `EXCEPT` 句を使って `station_name` 列を除外します。クエリによって station_name ごとに個別に行が作成されますが、SELECT ステートメントにより特徴だけが抽出されます。

`num_clusters` オプションを省略すると、BigQuery ML がトレーニング データの総行数から適切なデフォルト値を選択します。また、ハイパーパラメータを調整して適切な数値を見つけることもできます。適切なクラスタ数を決定するには、複数の `num_clusters` 値で CREATE MODEL クエリを実行して、誤差測定値を確認します。そして、誤差測定値が最小になるポイントを選択します。<br>
`ML.EVALUATE` を使用して評価を行うと、誤差測定値の `Davies–Bouldin` インデックスが表示されます。

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE (MODEL `bqml_tutorial_eu.london_station_clusters`)

BigQuery コンソールから `bqml_tutorial_eu` データセットを選択し、 `london_station_clusters` モデルを確認しましょう<br>
`EVALUATION` タブから、各クラスタの`centroid` を確認することができます。

<img src="img/centroid.png">

<h2>ステップ 4: ML.PREDICT 関数を使用してステーションのクラスタを予測する。</h2>

特定のステーションが属するクラスタを特定するには、ML.PREDICT 関数を使用します。次のクエリは、名前に「Kennington」という文字列を含むすべてのステーションのクラスタを予測します。

In [ ]:
%%bigquery
WITH
  hs AS (
  SELECT
    h.start_station_name AS station_name,
    h.duration,
    ST_DISTANCE(ST_GEOGPOINT(s.longitude,
        s.latitude),
      ST_GEOGPOINT(-0.1,
        51.5))/1000 AS distance_from_city_center
  FROM
    `bigquery-public-data.london_bicycles.cycle_hire` AS h
  JOIN
    `bigquery-public-data.london_bicycles.cycle_stations` AS s
  ON
    h.start_station_id = s.id
  WHERE
    h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
  stationstats AS (
  SELECT
    station_name,
    AVG(duration) AS duration,
    COUNT(duration) AS num_trips,
    MAX(distance_from_city_center) AS distance_from_city_center
  FROM
    hs
  GROUP BY
    station_name )
SELECT
  * EXCEPT(nearest_centroids_distance)
FROM
  ML.PREDICT( MODEL bqml_tutorial_eu.london_station_clusters,
    (
    SELECT
      *
    FROM
      stationstats
    WHERE
      REGEXP_CONTAINS(station_name, 'Kennington')))


<h3>クエリの詳細</h3>

このクエリは REGEXP_CONTAINS 関数を使用して、`station_name` 列に文字列「Kennington」を含むすべてのエントリを検索します。ML.PREDICT 関数は、これらの値を使用して、どのクラスタにステーションが含まれるかを予測します。

<h2>ステップ 5: モデルを使用してデータドリブンの意思決定を行う</h2>

評価結果により、さまざまなクラスタの状況を把握しやすくなります。

<img src="img/centroid.png">

この例では、`Cluster#2` は、市中心部付近の混雑している市内のステーションを表示します。<br>
`Cluster#1` は、より混雑度の低い 2 つ目の市内のステーションを表示します。<br>
`Cluster#4` は、より混雑度が低く、よりレンタル時間が長い、郊外のサブステーションを表示します。<br>
`Cluster#3` は、よりレンタル時間が短い、もう 1 つの郊外のサブステーションを表示します。<br>

以上のような結果を元にして、判断の根拠を示すためにデータを使用できます。例:

- 新しいタイプの鍵を試す必要があるとします。実験の対象には、どのクラスタのステーションを選択すればいいでしょうか。Cluster#1、Cluster#3、Cluster#4 は、最も混雑したステーションではないので、妥当な選択肢のようです。
- 長距離用の自転車をいくつかのステーションに置いてみたいと考えているとします。どのステーションが適しているでしょうか。Cluster#4 は、市内中心部から離れた場所にあるステーションのグループで、利用距離も最も長くなっています。したがって、長距離用の自転車を置くには最も適したステーションと思われます。

---

## ラボ: iris の種類を予測するクラスタリングモデルを作成する

`bigquery-public-data.ml_datasets.iris` データを使用して、iris の種類をクラスタリングするモデルを作成してください。<br>
iris の種類はversicolor, virginica, setosa の三種類です。（その際、正解の `species` 列は使用しないでください）

In [ ]:
%%bigquery iris
SELECT *,
  CASE
    WHEN MOD(ABS(FARM_FINGERPRINT(CAST(sepal_length AS STRING))), 10) < 8 THEN 'training'
    WHEN MOD(ABS(FARM_FINGERPRINT(CAST(sepal_length AS STRING))), 10) >= 8 THEN 'prediction'
  END AS dataframe
FROM `bigquery-public-data.ml_datasets.iris`

In [ ]:
iris

### モデルを作成する


モデルができたら、BigQuery コンソールから、モデルの詳細を確認してください

## モデルを使用して推論を行う

Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

Source: [https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial](https://cloud.google.com/bigquery-ml/docs/kmeans-tutorial)